In [1]:
###
# Poligonos de ee
###

# Módulos
import ee
import folium
import geehydro

# Inicio de API
ee.Initialize()

# region
polygonCollection = 'WCMC/WDPA/current/polygons'
polygonName = 'Cerro Saroche'#'Canaima National Park'#'Guatopo'

# poligono
polygon = ee.FeatureCollection(polygonCollection) \
            .filter(ee.Filter.eq('NAME', polygonName))

#buffer_poligon = 1000
google_poligon = ee.Geometry( polygon.geometry() )\
    .bounds()\
    .coordinates()\
    .get(0)\
    .getInfo()

latitude = list(map(lambda x: x[0] ,google_poligon))
longitude = list(map(lambda x: x[1] ,google_poligon))

In [8]:
###
# Datos de cdsapi
###

import cdsapi
import numpy as np

# buscando datos
year_start=1970
year_end=2021
months = ['01', '02', '03','04', '05', '06','07', '08', '09','10', '11', '12']
variable = ['total_precipitation']#['2m_temperature']#['total_precipitation']
directorio = './data/cdsapi/{variable}'.format(variable=variable[0])

# api cdsapi
# https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land-monthly-means?tab=overview
cds = cdsapi.Client("https://cds.climate.copernicus.eu/api/v2",
                    "40779" + ":" + "142accd6-9497-45c2-b607-de990bd8727c")
cds.retrieve('reanalysis-era5-land-monthly-means',
                {
                    'format': 'grib',
                    'product_type': 'monthly_averaged_reanalysis',
                    'variable': variable ,
                    'area': np.round([max(longitude), min(latitude), min(longitude),max(latitude)],2).tolist(),
                    'month': months,
                    'year': list(map(lambda x: str(x),list(range(year_start,year_end+1)) )),
                    'time': '00:00',
                },
                '{directorio}_cdsapi.grib'.format(directorio=directorio))

/media/javier/Compartida/doctorado/GEE-manual/gee-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
2021-07-17 10:20:53,629 INFO Welcome to the CDS
2021-07-17 10:20:53,631 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land-monthly-means
/media/javier/Compartida/doctorado/GEE-manual/gee-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
2021-07-17 10:20:53,936 INFO Request is queued
/media/javier/Compartida/doctorado/GEE-manual

Result(content_length=116160,content_type=application/x-grib,location=https://download-0011.copernicus-climate.eu/cache-compute-0011/cache/data3/adaptor.mars.internal-1626531653.966931-21473-2-4773479c-c8ad-407f-884e-bf60d43910ed.grib)

In [12]:

###
# Lectura de los datos
###
import pandas as pd
import pygrib

# funcion lectura de grib
def grib_data(grib):

    import numpy as np
    import pandas as pd

    latitude, longitude = grib.latlons()
    data_values = grib.values

    data_pandas = pd.DataFrame(
                                list(zip(np.round(latitude.reshape(1, latitude.size),2)[0],
                                    np.round(longitude.reshape(1, longitude.size),2)[0],
                                    data_values.reshape(1,data_values.size)[0]
                                    )),
                            columns=['latitude','longitude',grib.name])

    data_pandas['validDate'] = grib.validDate
    data_pandas['analDate'] = grib.validDate 
    data_pandas['id'] = data_pandas.index 
    data_pandas['year_month'] = pd.to_datetime(data_pandas['validDate']).dt.strftime("%Y%m")

    return data_pandas.sort_values(['validDate','id'],ascending=True)


In [13]:
# cargando la data precipitacion 
directorio = './data/cdsapi/{variable}_cdsapi.grib'.format(variable='total_precipitation')
precipitacion = pygrib.open(directorio)
precipitacion = pd.concat(list(map(grib_data, [x for x in precipitacion])))

In [14]:

precipitacion.head()

,latitude,longitude,Total precipitation,validDate,analDate,id,year_month
0,10.26,-69.88,0.000109,1981-01-01,1981-01-01,0,198101
1,10.26,-69.78,0.000114,1981-01-01,1981-01-01,1,198101
2,10.26,-69.68,0.000132,1981-01-01,1981-01-01,2,198101
3,10.26,-69.58,0.000161,1981-01-01,1981-01-01,3,198101
4,10.26,-69.48,0.000192,1981-01-01,1981-01-01,4,198101


In [7]:
print(precipitacion.year_month.min())
print(precipitacion.year_month.max())

198101
202104


In [15]:
precipitacion.shape

(7260, 7)

In [16]:
# cargando la data precipitacion 
directorio = './data/cdsapi/{variable}_cdsapi.grib'.format(variable='2m_temperature')
temperatura = pygrib.open(directorio)
temperatura = pd.concat(list(map(grib_data, [x for x in temperatura])))

temperatura.head()

,latitude,longitude,2 metre temperature,validDate,analDate,id,year_month
0,10.26,-69.88,298.865784,1981-01-01,1981-01-01,0,198101
1,10.26,-69.78,298.049866,1981-01-01,1981-01-01,1,198101
2,10.26,-69.68,297.177490,1981-01-01,1981-01-01,2,198101
3,10.26,-69.58,296.599792,1981-01-01,1981-01-01,3,198101
4,10.26,-69.48,295.995483,1981-01-01,1981-01-01,4,198101


In [17]:
print(temperatura.year_month.min())
print(temperatura.year_month.max())

198101
202104


In [18]:
temperatura.shape

(7260, 7)

In [19]:
data_final = precipitacion.merge(temperatura[['latitude','longitude','validDate','2 metre temperature']],left_on=['latitude','longitude','validDate'], right_on=['latitude','longitude','validDate'],how='inner')
data_final = data_final[['latitude',
                        'longitude',
                        'id',
                        'Total precipitation',
                        '2 metre temperature',
                        'year_month',
                        'validDate',
                        'analDate'
                        ]].copy()

data_final.head()

,latitude,longitude,id,Total precipitation,2 metre temperature,year_month,validDate,analDate
0,10.26,-69.88,0,0.000109,298.865784,198101,1981-01-01,1981-01-01
1,10.26,-69.78,1,0.000114,298.049866,198101,1981-01-01,1981-01-01
2,10.26,-69.68,2,0.000132,297.177490,198101,1981-01-01,1981-01-01
3,10.26,-69.58,3,0.000161,296.599792,198101,1981-01-01,1981-01-01
4,10.26,-69.48,4,0.000192,295.995483,198101,1981-01-01,1981-01-01


In [20]:
###
# Buscando elevacion
###

# creando lista de centroides
lista_=ee.List(data_final[data_final.validDate==data_final.validDate.max()][['longitude','latitude']].to_numpy().tolist())
centroides= ee.FeatureCollection(lista_.map( lambda x: ee.Feature(ee.Geometry.Point(x,proj='EPSG:4326'),{'id':lista_.indexOf(x)}) ))

# evaluacion
lista_estaciones = [[-69.88, 10.26],
                    [-69.78, 10.26],
                    #[-69.68, 10.26],
                    #[-69.58, 10.26],
                    #[-69.48, 10.26],
                    #[-69.88, 10.16],
                    [-69.78, 10.16],
                    [-69.68, 10.16],
                    [-69.58, 10.16],
                    [-69.48, 10.16],
                    #[-69.88, 10.06],
                    #[-69.78, 10.06],
                    [-69.68, 10.06],
                    #[-69.58, 10.06],
                    #[-69.48, 10.06]
                    ]
lista_eva_ = ee.List(lista_estaciones)
centroides_eva = ee.FeatureCollection(lista_eva_.map( lambda x: ee.Feature(ee.Geometry.Point(x,proj='EPSG:4326'),{'id':lista_eva_.indexOf(x)}) ))


# Elevacion
elevation = ee.Image('CGIAR/SRTM90_V4').select('elevation')
slope = ee.Terrain.slope(elevation).clip(polygon)
centro = polygon.geometry().centroid().coordinates().getInfo()[::-1]

# calculando el punto con la elevacion
reducer = ee.Reducer.median()
centroids_elevation = elevation.reduceRegions( ee.FeatureCollection(centroides), reducer.setOutputs(['elevation']), 90)

def info_elevation(data):
    return data.get('geometry').get('coordinates') + [data.get('properties').get('elevation'),data.get('properties').get('id')]

elevacion = pd.DataFrame(list(map(info_elevation,centroids_elevation.getInfo().get('features'))),columns=['longitude','latitude','elevation','id'])

In [21]:
# mean NDVI in the Xingu Park
Map = folium.Map(location=centro,zoom_start=11,control_scale =True,width='100%',height='100%')
centroides_eva

Map.addLayer(polygon.geometry().bounds(),{'color':'#E9EFE7'}, 'poligono')
Map.addLayer(slope, {min: 0, max: 60}, 'Slope')
Map.addLayer(centroides,{'color':'#7B7E79'}, 'centroides')
Map.addLayer(centroides_eva,{'color':'#FF0000'}, 'centroides_estaciones')

Map.setControlVisibility(layerControl=True,fullscreenControl=True,latLngPopup=True) # Layer control
Map

In [23]:
data_final_elevacion = data_final.merge(elevacion[['latitude','longitude','elevation']],left_on=['latitude','longitude'], right_on=['latitude','longitude'],how='left')
data_final_elevacion = data_final_elevacion[['latitude',
                                            'longitude',
                                            'id',
                                            'elevation',
                                            'Total precipitation',
                                            '2 metre temperature',
                                            'year_month',
                                            'validDate',
                                            'analDate'
                                            ]].copy()

data_final_elevacion.head()

,latitude,longitude,id,elevation,Total precipitation,2 metre temperature,year_month,validDate,analDate
0,10.26,-69.88,0,520,0.000109,298.865784,198101,1981-01-01,1981-01-01
1,10.26,-69.78,1,881,0.000114,298.049866,198101,1981-01-01,1981-01-01
2,10.26,-69.68,2,674,0.000132,297.177490,198101,1981-01-01,1981-01-01
3,10.26,-69.58,3,734,0.000161,296.599792,198101,1981-01-01,1981-01-01
4,10.26,-69.48,4,663,0.000192,295.995483,198101,1981-01-01,1981-01-01


In [24]:
# seleccionando solo estaciones
data_estaciones = data_final_elevacion.merge(pd.DataFrame(lista_estaciones,columns=['longitude','latitude']),
                                                            left_on=['latitude','longitude'],
                                                            right_on=['latitude','longitude'],
                                                            how='inner').sort_values(['id','year_month'],ascending=True)
data_estaciones

,latitude,longitude,id,elevation,Total precipitation,2 metre temperature,year_month,validDate,analDate
0,10.26,-69.88,0,520,0.000109,298.865784,198101,1981-01-01,1981-01-01
1,10.26,-69.88,0,520,0.001188,298.557373,198102,1981-02-01,1981-02-01
2,10.26,-69.88,0,520,0.000160,299.911194,198103,1981-03-01,1981-03-01
3,10.26,-69.88,0,520,0.006743,297.600708,198104,1981-04-01,1981-04-01
4,10.26,-69.88,0,520,0.004509,297.993896,198105,1981-05-01,1981-05-01
...,...,...,...,...,...,...,...,...,...
3383,10.06,-69.68,12,652,0.000232,297.207520,202012,2020-12-01,2020-12-01
3384,10.06,-69.68,12,652,0.000249,297.084351,202101,2021-01-01,2021-01-01
3385,10.06,-69.68,12,652,0.000186,297.539673,202102,2021-02-01,2021-02-01
3386,10.06,-69.68,12,652,0.000814,297.021423,202103,2021-03-01,2021-03-01


In [25]:
data_estaciones.shape

(3388, 9)

In [26]:
print(data_estaciones.year_month.min())
print(data_estaciones.year_month.max())

198101
202104


In [27]:
data_estaciones.id.unique()

array([ 0,  1,  6,  7,  8,  9, 12])

In [28]:
# guardando la data para los modelos
data_estaciones.to_pickle('./data/cdsapi/Cerro_Saroche_cdsapi.pkl')

In [29]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create traces
fig = make_subplots(rows=2, cols=4)
fig.add_trace(go.Scatter(x=data_estaciones[data_estaciones.id==0].year_month.tolist(),
                         y=data_estaciones[data_estaciones.id==0]['Total precipitation'].tolist(),
                         mode='lines',
                         name='lines'),
                         row=1, col=1)
fig.add_trace(go.Scatter(x=data_estaciones[data_estaciones.id==1].year_month.tolist(),
                        y=data_estaciones[data_estaciones.id==1]['Total precipitation'].tolist(),
                        mode='lines',
                        name='lines'),
                        row=1, col=2)
fig.add_trace(go.Scatter(x=data_estaciones[data_estaciones.id==6].year_month.tolist(),
                         y=data_estaciones[data_estaciones.id==6]['Total precipitation'].tolist(),
                         mode='lines',
                         name='lines'),
                         row=1, col=3)
fig.add_trace(go.Scatter(x=data_estaciones[data_estaciones.id==7].year_month.tolist(),
                         y=data_estaciones[data_estaciones.id==7]['Total precipitation'].tolist(),
                         mode='lines',
                         name='lines'),
                         row=1, col=4)
fig.add_trace(go.Scatter(x=data_estaciones[data_estaciones.id==8].year_month.tolist(),
                         y=data_estaciones[data_estaciones.id==8]['Total precipitation'].tolist(),
                         mode='lines',
                         name='lines'),
                         row=2, col=1)
fig.add_trace(go.Scatter(x=data_estaciones[data_estaciones.id==9].year_month.tolist(),
                         y=data_estaciones[data_estaciones.id==9]['Total precipitation'].tolist(),
                         mode='lines',
                         name='lines'),
                         row=2, col=2)
fig.add_trace(go.Scatter(x=data_estaciones[data_estaciones.id==12].year_month.tolist(),
                         y=data_estaciones[data_estaciones.id==12]['Total precipitation'].tolist(),
                         mode='lines',
                         name='lines'),
                         row=2, col=3)


fig.show()
